In [ ]:
import os
import sys

sys.path.append('..')

from movies_dask_bag.movie_reader import TheatersReader, MoviesReader, ShowingsReader

work_dir = os.environ.get('SLURM_TMPDIR', '.')
data_dir = '{}/json'.format(work_dir)
file_pattern = '{}/20191206/*'.format(data_dir)

theaters_reader = TheatersReader(file_pattern)

theaters_reader.take(1)

In [ ]:
theaters_reader.client

In [ ]:
%%time

theaters_reader.count

In [ ]:
def flatten_theaters_for_location(x):
    return {
        'lat': float(x['theater_lat']),
        'lon': float(x['theater_lon']),
        'country': x['country'],
        'hover': "{} (id: {}): {}".format(x['theater_name'],
                                      x['theater_id'],
                                      x['source_xml']) 
    }

location = theaters_reader.bag.map(flatten_theaters_for_location)
out = location.compute()

out[0]

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(out)
fig = px.scatter_geo(df, lat='lat', lon='lon', color='country',
                     hover_name='hover')
fig.show()

In [ ]:
import json
from shapely.geometry import shape, Point
import os, urllib.request
import pandas as pd

country_code = 'swe'
country = location.filter(lambda x:x['country'] == 'swe')
country_theaters = country.compute()
country_df = pd.DataFrame(country_theaters)

In [ ]:
border = '{}.geojson'.format(country_code.upper())
url = 'https://github.com/AshKyd/geojson-regions/raw/master/countries/10m/{}'.format(border)
output = 'data/{}'.format(border)

urllib.request.urlretrieve(url, output)

import json
from shapely.geometry import shape, Point

with open(output) as f:
    js = json.load(f)

polygon = shape(js['geometry'])

In [ ]:
country_df['in_boundary'] = country_df.apply(lambda x: polygon.distance(Point(x['lon'], x['lat'])) < 0.06, axis=1)

fig = px.scatter_geo(country_df, lat='lat', lon='lon', color='in_boundary',
                     hover_name='hover')
fig.show()

In [ ]:
country_df['in_boundary'].value_counts()

In [ ]:
theaters_reader.client.close()